In [1]:
import torch
import os
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
print(f"Using device: {device}")

Using device: cuda


In [3]:
pip install -qU llama-index llama-index-embeddings-huggingface llama-parse llama-index-llms-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.2/180.2 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.2/853.2 kB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/4

In [12]:
# LlamaParse is async-first, so we need to run this line of code if you are working on a notebook
import nest_asyncio

nest_asyncio.apply()
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_parse import LlamaParse
from llama_index.llms.huggingface import HuggingFaceInferenceAPI, HuggingFaceLLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

parser = LlamaParse(api_key=#insertkey,
                    result_type="text",
                    parsing_instruction = "The provided document is a chemical enginnering book. It contains tables, equations and diagrams. Output any math or chemical equation in LATEX markdown (between $$)",
                    show_progress = True,
                    skip_diagonal_text=True)
documents = await parser.aload_data("/content/chapter1.pdf")



Started parsing the file under job_id 9e590cdd-3099-4235-91be-734001d484a3


In [22]:
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

embeddings = Settings.embed_model.get_text_embedding("It is raining cats and dogs here!") #test



In [23]:
embeddings

[0.013021483086049557,
 -0.031699515879154205,
 0.09952757507562637,
 -0.014436560682952404,
 0.0496964193880558,
 -0.002786919940263033,
 0.0547209158539772,
 0.023421889171004295,
 -0.006272871512919664,
 -0.02143820747733116,
 -0.05713886767625809,
 -0.011216753162443638,
 -0.02090391144156456,
 0.0025230650790035725,
 -0.0035405075177550316,
 0.08258672803640366,
 0.0354522205889225,
 0.003919221926480532,
 0.06545630842447281,
 -0.006522278767079115,
 -0.022985996678471565,
 -0.06200127303600311,
 -0.07006347179412842,
 0.029751738533377647,
 0.042658738791942596,
 -0.04402843862771988,
 0.04222099855542183,
 0.03287284076213837,
 -0.024911323562264442,
 0.009935400448739529,
 0.04624434560537338,
 0.035239942371845245,
 -0.028916027396917343,
 -0.01899091899394989,
 0.014000823721289635,
 -0.0010815488640218973,
 -0.008744621649384499,
 -0.01392766460776329,
 -0.03017149306833744,
 -0.02275998517870903,
 -0.016626859083771706,
 0.01356144342571497,
 0.014432014897465706,
 0.02290

In [ ]:
Settings.llm = LMStudio(
    model_name="Hermes-2-Pro-Llama-3-8B",
    base_url="http://localhost:1234/v1",
    temperature=0.7,
)

index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()
response = query_engine.query("What is the first article?")
print(response)